In [1]:
# Deep Learning Assignment 2

# Import modules
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '/Users/rgparekh/Documents/Personal/Rajesh/Data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
# Reshape dataset into a flat vector X and the labels using the 1-hot encoding
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# Training a logistic regression classifier using TensorFlow

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
# Let's run the Tensor Flow graph performing GD

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 14.356435
Training accuracy: 15.3%
Validation accuracy: 19.1%
Loss at step 100: 2.358888
Training accuracy: 71.7%
Validation accuracy: 70.9%
Loss at step 200: 1.873673
Training accuracy: 75.2%
Validation accuracy: 73.2%
Loss at step 300: 1.619737
Training accuracy: 76.4%
Validation accuracy: 74.2%
Loss at step 400: 1.451404
Training accuracy: 77.1%
Validation accuracy: 74.5%
Loss at step 500: 1.328584
Training accuracy: 77.8%
Validation accuracy: 74.7%
Loss at step 600: 1.233152
Training accuracy: 78.5%
Validation accuracy: 74.9%
Loss at step 700: 1.155747
Training accuracy: 79.1%
Validation accuracy: 75.2%
Loss at step 800: 1.091101
Training accuracy: 79.6%
Validation accuracy: 75.4%
Test accuracy: 82.7%


In [9]:
# Stochastic Gradient Descent (SGD)

batch_size = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
# Run the SGD network

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.369888
Minibatch accuracy: 11.3%
Validation accuracy: 16.1%
Minibatch loss at step 500: 1.668026
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.174448
Minibatch accuracy: 75.4%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 1.247395
Minibatch accuracy: 77.0%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.150578
Minibatch accuracy: 78.1%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 1.124744
Minibatch accuracy: 73.8%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.934941
Minibatch accuracy: 74.6%
Validation accuracy: 78.5%
Minibatch loss at step 3500: 0.819073
Minibatch accuracy: 82.8%
Validation accuracy: 79.0%
Minibatch loss at step 4000: 0.815923
Minibatch accuracy: 77.0%
Validation accuracy: 79.2%
Minibatch loss at step 4500: 0.768683
Minibatch accuracy: 83.2%
Validation accuracy: 79.4%
Minibatch loss at step 5000: 0.893613
Minibatch accuracy: 79.3%
Validation accura

In [38]:
# Construct a multi-layer perceptron

# Constants
image_size = 28
num_labels = 10

# Parameters
learning_rate = 0.01
batch_size = 256
display_step = 1

graph = tf.Graph()
with graph.as_default():

    # Network Parameters
    n_hidden_1 = 128 # Hidden layer 1
    n_hidden_2 = 64 # Hidden layer 2 

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                  shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Create model
    def multilayer_perceptron(x, weights, biases):
        # Hidden layer 1 with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer 2 with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([image_size*image_size, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_labels]))
    }

    # Construct model
    pred = multilayer_perceptron(tf_train_dataset, weights, biases)

    # Define loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels))
    # GradientDescentOptimizer is the standard GD
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    #AdamOptimizer is an optimized version of GD
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(pred)
    valid_prediction = tf.nn.softmax(multilayer_perceptron(valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(multilayer_perceptron(test_dataset, weights, biases))

# Run the network
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 1154.441040
Minibatch accuracy: 7.8%
Validation accuracy: 11.1%
Minibatch loss at step 500: 2.015767
Minibatch accuracy: 41.8%
Validation accuracy: 41.6%
Minibatch loss at step 1000: 1.090208
Minibatch accuracy: 60.2%
Validation accuracy: 59.3%
Minibatch loss at step 1500: 1.690065
Minibatch accuracy: 65.2%
Validation accuracy: 64.8%
Minibatch loss at step 2000: 0.880495
Minibatch accuracy: 71.5%
Validation accuracy: 69.2%
Minibatch loss at step 2500: 1.241414
Minibatch accuracy: 69.9%
Validation accuracy: 71.3%
Minibatch loss at step 3000: 0.933003
Minibatch accuracy: 65.2%
Validation accuracy: 72.9%
Minibatch loss at step 3500: 0.718066
Minibatch accuracy: 78.9%
Validation accuracy: 75.0%
Minibatch loss at step 4000: 0.713917
Minibatch accuracy: 78.1%
Validation accuracy: 76.2%
Minibatch loss at step 4500: 0.562282
Minibatch accuracy: 80.5%
Validation accuracy: 77.0%
Minibatch loss at step 5000: 0.828531
Minibatch accuracy: 75.8%
Validation accur